In [ ]:
!pip install langchain langchain-groq langchain-community langchain-text-splitters sentence-transformers faiss-cpu pypdf python-dotenv

In [1]:

import streamlit as st
import os
import tempfile
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

c:\Users\PC\Desktop\VS code File\Rag_groq\rag_croq_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:

# Load PDF
loader = PyPDFLoader('C:/Users/PC/Desktop/VS code File/RAG_GROQ/RL exam 25-26.pdf')
docs = loader.load()

#if you want to use the web scrapping 
#loader = WebBaseLoader(website_url)
#docs = loader.load()
            
            
# Split text
splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000, 
                chunk_overlap=200
            )
chunks = splitter.split_documents(docs)
    
            
# Create embeddings
embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )

C:\Users\PC\AppData\Local\Temp\ipykernel_18748\3734642754.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [5]:
# Let's define the query, since we are going to use it multiple times.
query = "Give me the main comptetence of mohamed"



db = FAISS.from_documents(chunks, embeddings)

In [6]:
retrieved_docs = db.similarity_search(query, k=3)
context = "\n\n".join([d.page_content for d in retrieved_docs])

 
# initialize the retriever
retriever = db.as_retriever(
search_type="mmr", #similarity
search_kwargs={'k': 4}
)


In [7]:
llm = ChatGroq(
                        model="llama-3.3-70b-versatile",  
                        temperature=0.3,
                        max_tokens=512
                    )

In [8]:
m=llm.invoke("resume this file in 2 ligne")

In [9]:
m

AIMessage(content='There is no file to resume. This conversation just started. If you provide a file or text, I can summarize it for you in 2 lines.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 42, 'total_tokens': 74, 'completion_time': 0.119644113, 'completion_tokens_details': None, 'prompt_time': 0.0019849, 'prompt_tokens_details': None, 'queue_time': 0.043722735, 'total_time': 0.121629013}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_f8b414701e', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b2741-82bf-72b0-bb32-d5bb83387870-0', usage_metadata={'input_tokens': 42, 'output_tokens': 32, 'total_tokens': 74})

In [10]:
# Create prompt
prompt = ChatPromptTemplate.from_template("""
Réponds à la question en te basant uniquement sur le contexte fourni.
Si tu ne trouves pas la réponse dans le contexte, dis "Je n'ai pas cette information dans le document."

Contexte:
{context}

Question: {question}

Réponse:""")

In [11]:
from langchain_core.runnables import RunnablePassthrough              

output_parser = StrOutputParser()

chain = (
{"context": retriever, "question": RunnablePassthrough()}
| prompt
| llm
| output_parser
)


In [12]:
response = chain.invoke("donne moi le resume de ca")
response

"Le document décrit un projet de création d'un modèle de langage léger (1-10 millions de paramètres) capable de réaliser des additions et des soustractions de petits entiers (jusqu'à 3-4 chiffres) à l'aide de l'apprentissage par renforcement. Le projet comporte plusieurs étapes :\n\n1. Création d'un jeu de données synthétiques d'expressions arithmétiques avec les résultats corrects.\n2. Pré-entraînement du modèle sur un objectif de prédiction de token suivant.\n3. Affinage du modèle à l'aide de l'apprentissage par renforcement pour améliorer la précision mathématique.\n4. Évaluation du modèle à l'aide de métriques telles que la précision sur l'addition et la soustraction, la performance en fonction de la longueur des nombres, etc.\n\nLe document fournit également des détails sur l'architecture du modèle, les outils et les frameworks à utiliser, ainsi que des extensions optionnelles pour améliorer le projet. L'objectif final est de créer un modèle capable de raisonner à un niveau de chi